In [ ]:
#LDA topic to wordcloud #module 6.1
import pandas as pd
import numpy as np
import PIL.Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt 
import spacy
import collections


nlp = spacy.load("en_core_sci_lg")#tokenize by en_core_sci_lg

def pd_to_nlp(df):
    text = ""
    for title in df:
        doc = nlp(title)
        for ent in doc.ents:#explore entities by nlp.ent
            text = text+" "+str(ent).lower()
    return text

def nlp_count_pd(text,stopwords):
    ls = text.split()
    ls1 = []
    for i in ls:
        if i not in stopwords:
            ls1.append(i)
    c = collections.Counter(ls1)
    l_key_value = [[i,j] for i,j in zip(c.keys(),c.values())]
    l_pd = pd.DataFrame(l_key_value)
    display(l_pd)
    #l_pd.to_csv("/content/drive/MyDrive/text mining related/wordcount(2017to2022).csv")
    return l_pd
 

with open("../Files/SupplementalTables/SupplementalTable3.txt", "r") as f:
    stopwords = [w.strip() for w in f]
print(stopwords)

mask_ary = np.zeros(shape=(400, 400))
for i in range(400):
    for j in range(400):
        if (i-200)**2 + (j-200)**2 > 180**2:
            mask_ary[i, j] = 255

df1 = pd.read_csv("../Files/Figure3/Table7.csv")
df1["period"]=1
df2 = pd.read_csv("../Files/Figure3/Table8.csv")
df2["period"]=2
df3 = pd.read_csv("../Files/Figure3/Table9.csv")
df3["period"]=3
df4 = pd.read_csv("../Files/Figure3/Table10.csv")
df4["period"]=4
df = pd.concat([df1,df2,df3,df4],axis=0)
df.drop_duplicates(subset="PMID",inplace=True)
df.set_index("PMID",inplace=True)
df.drop(columns=["period"], inplace=True)
display(df)

df_lda = pd.read_csv("../Files/Figure4/Table32.csv",index_col=0)
display(df_lda)

df_merge = pd.concat([df,df_lda],axis=1)
display(df_merge)

for i,period in zip([1,2,3,4],["I","II","III","IV"]):
    for j, topic in zip(range(19),["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"]):
        df_comp = df_merge[(df_merge["period"]==i)&(df_merge["topic"]==j)]
        df_ta = df_comp["title"]
        display(df_ta)
        text = pd_to_nlp(df_ta)
        nlp_count_pd(text,stopwords)
        print(text)
        wordcloud = WordCloud(background_color="white",stopwords=stopwords,mask=mask_ary).generate(text)
        #collocations +
        plt.imshow(wordcloud)
        wordcloud.to_file("../Files/SupplementalFigures/SupplementalFigure3/WC(period{}_topic{}).png".format(period,topic))